# Install

# Import

In [1]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import re,os,json
import logging
logging.getLogger('pdfminer.pdfinterp').setLevel(logging.ERROR)
logging.getLogger('pdfminer.layout').setLevel(logging.ERROR)

# fn

In [2]:
from modules.extract_logic import clean_row, extract_sections, apply_dynamic_headers
from modules.schedule import ITR1Sections,ITR1BatchProcessor,PDFPipeline

In [3]:
input_file_path = 'OUTPUT/ ITR 1-2 AY 2025-26.pdf'
output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
config_path = "config/ITR1_parts.json"
itr = ITR1Sections(input_file_path, output_file_path, config_path)
itr

In [4]:
itr.config

{'PART_B': {'table_start_ptr': 'PART B GROSS TOTAL INCOME',
  'hdr_row_map': ['B1'],
  'ftr_row_map': 'Gross Total Income',
  'indentation': 'B'}}

In [5]:
print(itr.start_patterns)
print(itr.end_patterns)
print(itr.hdr_map)

{'PART_B': 'PART B GROSS TOTAL INCOME'}
{'PART_B': 'Gross Total Income'}
{'PART_B': ['B1']}


In [6]:
itr.sections

{'PART_B': {'start': 22, 'end': 68}}

In [7]:
def clean_and_prepend_none(data_slice,startswith):
    cleaned_slice = []
    for sublist in data_slice:
        cleaned_sublist = [item.replace('\n', ' ') if isinstance(item, str) else item for item in sublist if item]    
        prepend_none = False
        if cleaned_sublist and isinstance(cleaned_sublist[0], str):
            if not cleaned_sublist[0].startswith(startswith):prepend_none = True
        if prepend_none:cleaned_slice.append([None] + cleaned_sublist)
        else:cleaned_slice.append(cleaned_sublist)
    return cleaned_slice

In [9]:
for name, meta in itr.sections.items():
    s, e = meta["start"], meta["end"]
    # if itr.config
    itr.extracted[s:e] = clean_and_prepend_none(itr.extracted[s:e],"B")
    df_raw = pd.DataFrame(itr.extracted[s:e])
    df_raw = clean_row(df_raw)
    # display(df_raw)

In [ ]:
df_raw.to_excel("del.xlsx",sheet_name=name,index=False)

In [ ]:
assert False

# Process ONE PDF

In [ ]:
input_file_path = 'OUTPUT/ ITR 1 AY 2023-24.pdf'
output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
config_path = "config/ITR1_header.json"
itr = ITR1Sections(input_file_path,output_file_path,config_path)
print(itr.pan)
print(itr.ack)

In [ ]:
itr.get_section("SCHEDULE_TDS1")

# Process ALL PDFs

In [ ]:
input_dir="OUTPUT"
config_path = "config/ITR1_header.json"
processor = ITR1BatchProcessor(input_dir, config_path)
processor.process_all()

In [ ]:
metadata_df = processor.metadata()
metadata_df

In [ ]:
itr1_0=metadata_df.loc[0, "itr_obj"]
itr1_0

In [ ]:
itr1_0.debug["section_ranges"]

In [ ]:
# processor.export_by_pan()